# Отчет о запусках
---

## Сводки о прогонах

1. Сводная информация о прогонах [кода](https://github.com/banzarkhan/motorika-internship-team/blob/main/baseline_khavi.ipynb) (алгоритм предсказания - SVM) по коллекциям данных ОМГ-датчиков.

In [83]:
import pandas as pd

df_meta = pd.read_csv('./data/meta_information.csv', index_col=0)

In [90]:
svm_inf_f1_weighted = [
    0.7778, 0.6128, 0.5937, 0.7149, 0.6651, 0.6960, 0.7355, 0.8330, 0.8992, 0.8713,
    0.8698, 0.8437, 0.8626, 0.9137, 0.9118, 0.9014, 0.8895, 0.9085, 0.8897, 0.5932,
    0.8795, 0.8614, 0.9159, 0.4454, 0.6807, 0.8477, 0.8405, 0.8718, 0.8871, 0.4075,
    0.6979
]

svm_worst_f1 = [
    0.0000, 0.0000, 0.0000, 0.1928, 0.1927, 0.3207, 0.0000, 0.6713, 0.8042, 0.7680,
    0.7600, 0.7512, 0.7082, 0.7759, 0.7527, 0.7979, 0.7799, 0.7351, 0.6818, 0.2985,
    0.5351, 0.7630, 0.8179, 0.0000, 0.2653, 0.6964, 0.6471, 0.7951, 0.7948, 0.2246,
    0.2147
]

svm_gesture_name = [
    'Thumb', 'Thumb', 'Thumb', 'Thumb', 'Thumb', 'Thumb', 'Thumb', 'OK', 'OK', 'Pistol',
    'Grab', 'Grab', 'Grab', 'Thumb', 'Thumb', 'Grab', 'Open', 'OK', 'Thumb', 'Thumb', 
    'Thumb', 'Open', 'Thumb', 'evrt. except Neutral', 'OK', 'OK', 'Pistol', 'Pistol', 'OK', 'OK', 
    'OK'
]

table1 = pd.concat(
    [
        df_meta[['montage', 'pilote_id', 'len(train)', 'len(test)']],
        pd.DataFrame({
            'inf f1_weighted': svm_inf_f1_weighted,
            'worst f1': svm_worst_f1,
            'gesture name': svm_gesture_name
            })
    ], axis=1)

table1

,montage,pilote_id,len(train),len(test),inf f1_weighted,worst f1,gesture name
0,2023-05-15_16-16-08.palm,1,23337,5810,0.7778,0.0000,Thumb
1,2023-05-15_17-12-24.palm,1,23336,5803,0.6128,0.0000,Thumb
2,2023-06-05_16-12-38.palm,1,17939,4431,0.5937,0.0000,Thumb
3,2023-06-05_17-53-01.palm,1,17771,4435,0.7149,0.1928,Thumb
4,2023-06-20_14-43-11.palm,1,17936,4441,0.6651,0.1927,Thumb
5,2023-06-20_13-30-15.palm,1,17928,4435,0.6960,0.3207,Thumb
6,2023-06-20_12-34-17.palm,1,17758,4444,0.7355,0.0000,Thumb
7,2023-09-30_08-06-44.palm,2,5693,5509,0.8330,0.6713,OK
8,2023-09-29_11-03-50.palm,2,5694,5511,0.8992,0.8042,OK
9,2023-09-29_09-20-47.palm,2,5690,5507,0.8713,0.7680,Pistol


2. Сводная информация о прогонах [кода](https://github.com/banzarkhan/motorika-internship-team/blob/main/baseline_khavi_nn.ipynb) (алгоритм предсказания - нейросеть) по коллекциям данных ОМГ-датчиков.

In [ ]:
# nn_inf_f1_weighted = [
#     
# ]

# nn_worst_f1_weighted = [
#     
# ]

# nn_gesture_name = [
#     
# ]

# table2 = pd.concat(
#     [
#         df_meta[['montage', 'pilote_id', 'len(train)', 'len(test)']],
#         pd.DataFrame({
#             'inf f1_weighted': nn_inf_f1_weighted,
#             'worst f1_weighted': nn_worst_f1_weighted,
#             'gesture name': nn_gesture_name
#             })
#     ], axis=1)

# table2

---
## Исследование гомогенности выборок

Цель — проверить однородность данных в трейн и тест выборках во всех сетах.

In [70]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [98]:
result = []

for i in range(len(df_meta)):
    
    # Путь к файлу
    data_path = './data/' + df_meta['montage'][i]
    
    # Загрузка данных
    data = pd.read_csv(data_path, sep=' ').to_numpy()[:, :50]
    
    # Индекс отсечения
    last_train_idx = df_meta['last_train_idx'][i]
    
    # Формирование вектора лейблов принадлежности к выборке
    target = np.array([1 if i <= last_train_idx else 0 for i in range(len(data))])
    
    # Воссоединение данных с таргетом для перемешивания
    data_with_target = np.concatenate((data, target.reshape(-1, 1)), axis=1)
    
    # Перемешивание
    np.random.shuffle(data_with_target)
    
    # Разделение данных и таргета
    shuffled_data = data_with_target[:, :-1]
    shuffled_target = data_with_target[:, -1]
    
    # Сэмплирование
    X_train, X_test, y_train, y_test = train_test_split(shuffled_data, shuffled_target, test_size=0.2, random_state=42)

    # Создание и обучение модели
    model = LinearSVC(dual='auto')
    model.fit(X_train, y_train)

    # Предсказание принадлежности строк данных к выборкам
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)
    
    # Расчет и сохранение метрик
    f1_train_class_0 = f1_score(y_train, train_preds, pos_label=0)
    f1_train_class_1 = f1_score(y_train, train_preds, pos_label=1)
    f1_test_class_0 = f1_score(y_test, test_preds, pos_label=0)
    f1_test_class_1 = f1_score(y_test, test_preds, pos_label=1)
    result.append([f1_train_class_0, f1_train_class_1, f1_test_class_0, f1_test_class_1])

In [103]:
result_df = pd.DataFrame({
    'datasets': list(df_meta['montage']),
    'pilote_id': list(df_meta['pilote_id']),
    'f1_train_class (train)': [el[1] for el in result],
    'f1_test_class (train)': [el[0] for el in result],
    'f1_train_class (test)': [el[-1] for el in result],
    'f1_test_class (test)': [el[-2] for el in result]
})

result_df = pd.concat([result_df, table1[['inf f1_weighted']]], axis=1)
result_df

,datasets,pilote_id,f1_train_class (train),f1_test_class (train),f1_train_class (test),f1_test_class (test),inf f1_weighted
0,2023-05-15_16-16-08.palm,1,0.905107,0.510110,0.904314,0.501597,0.7778
1,2023-05-15_17-12-24.palm,1,0.942799,0.747620,0.947079,0.771128,0.6128
2,2023-06-05_16-12-38.palm,1,0.966862,0.857692,0.968880,0.872000,0.5937
3,2023-06-05_17-53-01.palm,1,0.977701,0.909221,0.975074,0.900618,0.7149
4,2023-06-20_14-43-11.palm,1,0.958255,0.826962,0.953142,0.799528,0.6651
5,2023-06-20_13-30-15.palm,1,0.928948,0.685185,0.924423,0.660528,0.6960
6,2023-06-20_12-34-17.palm,1,0.948294,0.780062,0.945592,0.772462,0.7355
7,2023-09-30_08-06-44.palm,2,0.871778,0.870628,0.864276,0.868802,0.8330
8,2023-09-29_11-03-50.palm,2,0.801215,0.804471,0.805593,0.809713,0.8992
9,2023-09-29_09-20-47.palm,2,0.920884,0.919206,0.928665,0.925740,0.8713


In [101]:
# вычисляем корреляцию между предсказаниями трейн-класса и
# средневзвешенными f1-мерами 
correlation = result_df['f1_train_class (train)'].corr(result_df['inf f1_weighted'])
print("Корреляция:", correlation)

# вычисляем корреляцию между предсказаниями тест-класса и
# средневзвешенными f1-мерами 
correlation = result_df['f1_test_class (train)'].corr(result_df['inf f1_weighted'])
print("Корреляция:", correlation)

# вычисляем корреляцию между предсказаниями трейн-класса и
# средневзвешенными f1-мерами 
correlation = result_df['f1_train_class (test)'].corr(result_df['inf f1_weighted'])
print("Корреляция:", correlation)

# вычисляем корреляцию между предсказаниями тест-класса и
# средневзвешенными f1-мерами 
correlation = result_df['f1_test_class (test)'].corr(result_df['inf f1_weighted'])
print("Корреляция:", correlation)

Корреляция: -0.38785545593211956
Корреляция: -0.48220030133241304
Корреляция: -0.36655772308354867
Корреляция: -0.47080832194802086


Полученный результат свидетельствует о наличии умеренной отрицательной корреляции между метриками предсказаний принадлежности строки данных к выборке (тестовой или треннировочной) и значениями метрик предсказания жестов.

Т.е. мы лишь с некоторой оговоркой можем утверждать, что чем хуже удалось предсказать принадлежность строки данных к выборке (тестовой или треннировочной), тем выше точность предсказания жестов. Различие в данных из тестовой и треннировочной выборок действительно может иметь место, но это, вероятно, не единственная причина плохих предсказаний жестов.